# Formant Frequencies

# Measuring Formants

## Import modules

In [60]:
import parselmouth
from parselmouth.praat import call

## Load the sound into Parselmouth-Praat

In [56]:
AUDIO_FILE="03-01-01-01-01-01-01.wav"
sound = parselmouth.Sound(AUDIO_FILE)
print(sound)

Object type: Sound
Object name: <no name>
Date: Mon Sep 25 11:02:01 2023

Number of channels: 1 (mono)
Time domain:
   Start time: 0 seconds
   End time: 3.3032916666666665 seconds
   Total duration: 3.3032916666666665 seconds
Time sampling:
   Number of samples: 158558
   Sampling period: 2.0833333333333333e-05 seconds
   Sampling frequency: 48000 Hz
   First sample centred at: 1.0416666666666666e-05 seconds
Amplitude:
   Minimum: -0.0384216309 Pascal
   Maximum: 0.0405883789 Pascal
   Mean: 9.14999977e-07 Pascal
   Root-mean-square: 0.00400063537 Pascal
Total energy: 5.28694585e-05 Pascal² sec (energy in air: 1.32173646e-07 Joule/m²)
Mean power (intensity) in air: 4.00127084e-08 Watt/m² = 46.02 dB
Standard deviation in channel 1: 0.00400064788 Pascal



In [57]:
from IPython.display import Audio
Audio(data=sound.values, rate=sound.sampling_frequency)

## Measure the formants

### Analysis parameters matter
- Maximum Formant
    - This is the highest frequency a formant could be
    - Children's voices - 7000 Hz, 8000 Hz
    - Adult voices
        - Lower pitched voices use 5000
        - Higher pitched voices use 5500
    - Our cutoff will be 120 Hz or about the average male voice pitch

In [54]:
formant_object = sound.to_formant_burg(maximum_formant=5000)
print(formant_object)

Object type: Formant
Object name: <no name>
Date: Mon Sep 25 11:01:15 2023

Time domain:
   Start time: 0 seconds
   End time: 3.3032916666666665 seconds
   Total duration: 3.3032916666666665 seconds
Time sampling:
   Number of frames: 521
   Time step: 0.00625 seconds
   First frame centred at: 0.026645833333333258 seconds



In [59]:
f1 = call(formant_object, "Get mean", 1, 0, 0, "Hertz")
f2 = call(formant_object, "Get mean", 2, 0, 0, "Hertz")
f3 = call(formant_object, "Get mean", 3, 0, 0, "Hertz")
f4 = call(formant_object, "Get mean", 4, 0, 0, "Hertz")
print(f'f1: {f1}')
print(f'f2: {f2}')
print(f'f3: {f3}')
print(f'f4: {f4}')

f1: 778.9608604712985
f2: 1791.351328285888
f3: 2732.705836993562
f4: 3699.7894249084575


In [ ]:
pitch = sound.to_pitch()
mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")
print(mean_pitch)

### `if` the voice pitch is low, use `5000`; `elif` the voice pitch is high, use `5500`

In [ ]:
def measure_formants(sound):
    pitch = sound.to_pitch()
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")
    if mean_pitch < 120:
        max_formant = 5000
    elif mean_pitch >= 120:
        max_formant = 5500
    else:
        max_formant = 5500  # It's always good to have a fallback value just in case
    return max_formant


In [ ]:
max_formant = measure_formants(sound)
formant_object = sound.to_formant_burg(maximum_formant=max_formant)
f1 = call(formant_object, "Get mean", 1, 0, 0, "Hertz")
f2 = call(formant_object, "Get mean", 2, 0, 0, "Hertz")
f3 = call(formant_object, "Get mean", 3, 0, 0, "Hertz")
f4 = call(formant_object, "Get mean", 4, 0, 0, "Hertz")
print(f'formant 1: {f1}')
print(f'f2: {f2}')
print(f'f3: {f3}')
print(f'f4: {f4}')

In [ ]:
measure_formants(sound)

# F1 F2 plots

## Import modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set() # Use seaborn's default style to make attractive graphs
plt.rcParams['figure.dpi'] = 100 # Show nicely large images in this notebook

## Get formant database from Praat

In [ ]:
peterson_barney = call("Create formant table (Peterson & Barney 1952)")
print(peterson_barney)
call(peterson_barney, "Save as comma-separated file", "peterson_barney.csv")
peterson_barney = pd.read_csv('peterson_barney.csv')

## View the top 5 rows of your data file using pandas

In [ ]:
peterson_barney.head()

## Convert Hz to Bark

Bark corresponds to the tonotopic map of the cochlea

In [ ]:
def hz_to_bark(hz):
    bark = 7 * np.log(hz/650 + np.sqrt(1 + (hz/650)**2))
    return bark

In [ ]:
peterson_barney['F1 Bark'] = hz_to_bark(peterson_barney['F1'])
peterson_barney['F2 Bark'] = hz_to_bark(peterson_barney['F2'])

In [ ]:
peterson_barney.head()

## This code is from `matplotlib` and will draw ellipses around our vowels

You don't need to know how this works, just know how to run it.  One cool thing about Python is being able to reuse other people's code without reinventing the wheel.

<img src="img/python_comic.png">

In [ ]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

## Make an F1 F2 plot

Here is the code in steps, so I can explain what each line does.  It won't really work unless it's in 1 code cell

### We are going to overlay the plot of each vowel on the figure, so we need to create sublots first.

In [ ]:
fig, ax_kwargs = plt.subplots()

### Make the ellipses grey

In [ ]:
ax_kwargs.axvline(c='grey', lw=1)
ax_kwargs.axhline(c='grey', lw=1)

### Make a list of colours so that each vowel is a different colour

In [ ]:
colours = ['g', 'r', 'y', 'k', 'b', 'm', 'w']

### Make a `set` of all the vowels
A set is like a list, except it contains only unique items.  We're doing this to get a set of unique values from the `Vowel` column in the `peterson_barney` dataframe.  That column has a lot more than 1 example for each vowel.
- You can use `union`, and `intersection` with sets

In [ ]:
vowels = set(peterson_barney["Vowel"])

### Make a `for` loop to plot F1 and F2 of each vowel
Also, plot those ellipses

In [ ]:
for vowel in vowels:
    x = peterson_barney['F1 Bark'][peterson_barney['Vowel'] == vowel] 
    y = peterson_barney['F2 Bark'][peterson_barney['Vowel'] == vowel]
    confidence_ellipse(x, y, ax_kwargs, n_std=2,
        alpha=0.3, facecolor='pink', edgecolor='black', zorder=0)
    ax_kwargs.scatter(x, y, s=3)

### Set the figure title

In [ ]:
ax_kwargs.set_title(f'F1 F2 plot')

### Change X and Y axis limits

In [ ]:
plt.ylim(bottom=4)
plt.xlim(left=0)

### Label the X and Y axes

In [ ]:
ax_kwargs.set_xlabel("F1 Bark")
ax_kwargs.set_ylabel("F2 Bark")

### Final adjustments and show the figure

In [ ]:
fig.subplots_adjust(hspace=0.25)
fig1 = plt.gcf()
plt.show()

### Here's all that code together with a working plot

In [ ]:
sns.set() # Use seaborn's default style to make attractive graphs
plt.rcParams['figure.dpi'] = 400 # Show nicely large images in this notebook

fig, ax_kwargs = plt.subplots()

ax_kwargs.axvline(c='grey', lw=1)
ax_kwargs.axhline(c='grey', lw=1)

colours = ['g', 'r', 'y', 'k', 'b', 'm', 'w']
vowels = set(peterson_barney["Vowel"])

for vowel in vowels:
    x = peterson_barney['F1 Bark'][peterson_barney['Vowel'] == vowel] 
    y = peterson_barney['F2 Bark'][peterson_barney['Vowel'] == vowel]
    confidence_ellipse(x, y, ax_kwargs, n_std=2,
        alpha=0.3, facecolor='pink', edgecolor='black', zorder=0)
    ax_kwargs.scatter(x, y, s=100, marker=f"${vowel}$")
ax_kwargs.set_title(f'F1 F2 plot')

plt.ylim(bottom=4)
plt.xlim(left=0)

ax_kwargs.set_xlabel("F1 Bark")
ax_kwargs.set_ylabel("F2 Bark")

fig.subplots_adjust(hspace=0.25)
fig1 = plt.gcf()
plt.grid(False)
plt.show()

# Plotting formant frequencies on a Spectrogram

## Re-use our spectrogram code from the previous labs

In [ ]:
def draw_spectrogram(spectrogram, dynamic_range=70, cmap='afmhot'):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap=cmap)
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

spectrogram = sound.to_spectrogram(window_length=0.05)
draw_spectrogram(spectrogram)

### Find the times at which we measured formants

In [ ]:
formants = sound.to_formant_burg(maximum_formant=5000)
sample_times = formants.xs()

### Measure intensity so we plot formants only where there is sound

The algorithm calculates formants even when they aren't there

In [ ]:
intensity = sound.to_intensity()
print(intensity)

### For each of the four formants do these things...

You can also use numbers in a for loop.  
The `range` function here does each operation from values starting at 1, and ending at 5, but not including 5.  
Each time we go through the loop, `i` gets incremented 1 higher, so in this loop it gets:  
1  
2  
3  
4

In [ ]:
for i in range(1, 5):
    formant_values = call(formants, "To Matrix", i).values[0, :]
    for j, time in enumerate(sample_times, 1):
        intensity_value = call(intensity, "Get value at time", time, "cubic")
        if intensity_value < 50:
            formant_values[j] = 0
        formant_values[formant_values == 0] = np.nan
    plt.scatter(sample_times, formant_values, c='w', linewidth=3, marker='o', s=2)
    plt.scatter(sample_times, formant_values, c='r', linewidth=1, s=2)
    plt.grid(False)

1) Convert formant values from Praat formant to `numpy` format.

2) Go through each formant measurement over time

The `enumerate` function in Python allows us to mix words and numbers in `for` loops.  
We are going to assign `j` one number higher, starting at `1` each time we go through the loop.  
We are going to assign each `sample_time` to the `time` variable each time we go through the loop.

#### Measure intensity at each time that we measured formants

#### If those intensity values are too soft, discard the formant measurement so we don't plot formants on empty sound

#### Plot the formant values in red with white background

#### Turn off grids

### Here's that code all together in 1 cell so it works nicely to produce a high-quality spectrogram

In [ ]:
spectrogram = sound.to_spectrogram(window_length=0.05)
draw_spectrogram(spectrogram)

sample_times = formants.xs()
intensity = sound.to_intensity()
for i in range(1,5):
    formant_values = call(formants, "To Matrix", i).values[0, :]
    for j, time in enumerate(sample_times, 1):
        intensity_value = call(intensity, "Get value at time", time, "cubic")
        if intensity_value < 50:
            formant_values[j] = 0
    formant_values[formant_values == 0] = np.nan
    plt.scatter(sample_times, formant_values, c='w', linewidth=3, marker='o', s=2)
    plt.scatter(sample_times, formant_values, c='r', linewidth=1, s=2)
    plt.grid(False)


In [ ]:
from playsound import playsound
playsound('audio.mp3')